## 159: Sistema de cambio de contraseña usando FastAPI y JWT

Implementar una ruta que permita a un usuario cambiar su contraseña de forma segura, siempre que esté autenticado mediante un token JWT válido.

🧩 Código completo con cambio de contraseña
python
Copiar
Editar
from fastapi import FastAPI, HTTPException, Depends
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker, Session
from passlib.context import CryptContext
from jose import jwt, JWTError
from datetime import datetime, timedelta

# --- Configuración base ---
app = FastAPI()
DATABASE_URL = "sqlite:///./usuarios.db"
SECRET_KEY = "CLAVE_ULTRA_SECRETA"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# --- Configuración de Base de Datos ---
Base = declarative_base()
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(bind=engine, autoflush=False)

class Usuario(Base):
    __tablename__ = "usuarios"
    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    contraseña = Column(String)
    rol = Column(String)

Base.metadata.create_all(bind=engine)

# --- Modelos Pydantic ---
class CambioContraseña(BaseModel):
    contraseña_actual: str
    nueva_contraseña: str

# --- Utilidades ---
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

def verificar_contraseña(plain: str, hashed: str) -> bool:
    return pwd_context.verify(plain, hashed)

def hashear_contraseña(contraseña: str) -> str:
    return pwd_context.hash(contraseña)

def obtener_usuario_actual(token: str = Depends(oauth2_scheme), db: Session = Depends(get_db)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        email = payload.get("sub")
        if email is None:
            raise HTTPException(status_code=401, detail="Token inválido")
        usuario = db.query(Usuario).filter(Usuario.email == email).first()
        if usuario is None:
            raise HTTPException(status_code=404, detail="Usuario no encontrado")
        return usuario
    except JWTError:
        raise HTTPException(status_code=401, detail="Token inválido")

# --- Ruta de cambio de contraseña ---
@app.post("/usuario/cambiar-contraseña")
def cambiar_contraseña(datos: CambioContraseña, usuario=Depends(obtener_usuario_actual), db: Session = Depends(get_db)):
    if not verificar_contraseña(datos.contraseña_actual, usuario.contraseña):
        raise HTTPException(status_code=400, detail="La contraseña actual no es correcta")
    usuario.contraseña = hashear_contraseña(datos.nueva_contraseña)
    db.commit()
    return {"mensaje": "Contraseña actualizada correctamente"}